In [2]:
import duckdb
import os
import dask as dd
import pandas as pd

In [3]:
con = duckdb.connect(database="pipeline_logs.db")
schemas = con.execute("SHOW TABLES").fetchall()
schemas
# TODO: VERIFY THAT [('pipeline_logs',), ('processing_stats',)] TABLES EXIST IF TEST IS TO PASS GOING FORWARD 

[('pipeline_logs',), ('processing_stats',)]

In [4]:
# disconnect from the database
con.close()

In [3]:
pipeline_logs_table = con.execute("SELECT * FROM pipeline_logs").fetchdf()
pipeline_logs_table.head()

,log_id,timestamp,level,logger_name,message,file_path,sheet_name,operation_type,duration_ms,rows_processed,success,error_message
0,4f2ad931-ee27-42cf-8b60-26330a877279,2025-09-18 16:45:40.908650,DEBUG,__main__,Counted 17567 rows in sheet 'nextech',C:\etl\data\input\sales\cash_invoices\summariz...,nextech,row_counting,<NA>,17567,<NA>,None
1,ead211e9-e6fc-4121-8792-e82edbbcdea4,2025-09-18 16:45:43.919689,INFO,__main__,Converted sheet to Parquet,C:\etl\data\input\sales\cash_invoices\summariz...,nextech,sheet_conversion,1917,17567,True,None
2,204b0eb4-0790-4e34-8a95-81ed5933ef87,2025-09-18 16:45:43.976616,INFO,__main__,Successfully processed Excel file,C:\etl\data\input\sales\cash_invoices\summariz...,None,file_processing_complete,<NA>,17567,True,None
3,0ac16732-9071-46ae-b0ba-04b8c4cf68ff,2025-09-18 16:45:46.270748,DEBUG,__main__,Counted 19028 rows in sheet 'nextech',C:\etl\data\input\sales\cash_invoices\summariz...,nextech,row_counting,<NA>,19028,<NA>,None
4,0cedf54d-5a22-450d-9bac-1e18cd8f02eb,2025-09-18 16:45:49.012006,INFO,__main__,Converted sheet to Parquet,C:\etl\data\input\sales\cash_invoices\summariz...,nextech,sheet_conversion,1663,19028,True,None


In [ ]:
processing_stats_table = con.execute("SELECT * FROM processing_stats").fetchdf()
processing_stats_table
# TODO: VERIFY THAT THE processing_stats_table HAS THE EXPECTED COLUMNS IF TEST IS TO PASS GOING FORWARD
# total_files	total_sheets	total_excel_rows	total_parquet_rows	total_duration_ms	success_count	failure_count	files_with_row_mismatch	total_row_difference
# MAKE SURE total_files, total_sheets, success_count ARE EQUAL
# MAKE SURE failure_count IS 0
# MAKE SURE files_with_row_mismatch IS 0
# MAKE SURE total_row_difference IS 0
# MAKE SURE total_excel_rows, total_parquet_rows ARE EQUAL

,run_id,start_time,end_time,total_files,total_sheets,total_excel_rows,total_parquet_rows,total_duration_ms,success_count,failure_count,files_with_row_mismatch,total_row_difference
0,3f9d1c15-1b8b-477d-95b8-b1138148bac0,2025-09-18 16:45:38.526417,2025-09-18 16:48:05.065702,11,11,371619,371619,146539,11,0,0,0


In [ ]:
row_count = con.execute("SELECT COUNT(*) FROM processing_stats").fetchone()[0]
row_count

In [ ]:
# TODO: MANUALLY VERIFY THAT THE ROW COUNT IS AS EXPECTED IF TEST IS TO PASS GOING FORWARD
import os
import pandas as pd
import dask.dataframe as dd

folder = r"C:\etl\data\input\sales\cash_invoices\detailed\2025\january"
files = [os.path.join(folder, f) for f in os.listdir(folder) if f.endswith('.xls')]

excel_results = []
for file in files:
    xls = pd.ExcelFile(file)
    sheet_count = len(xls.sheet_names)
    for sheet in xls.sheet_names:
        df = dd.from_pandas(pd.read_excel(file, sheet_name=sheet, header=0), npartitions=1)
        col_count = len(df.columns)
        row_count = df.shape[0].compute()
        excel_results.append({
            'file': os.path.basename(file),
            'sheet': sheet,
            'sheet_count': sheet_count,
            'column_count': col_count,
            'row_count': row_count
        })

excel_results

[{'file': 'detci_01_05_january_2025.xls',
  'sheet': 'nextech',
  'sheet_count': 1,
  'column_count': 31,
  'row_count': 45226},
 {'file': 'detci_06_08_january_2025.xls',
  'sheet': 'nextech',
  'sheet_count': 1,
  'column_count': 31,
  'row_count': 42422},
 {'file': 'detci_09_11_january_2025.xls',
  'sheet': 'nextech',
  'sheet_count': 1,
  'column_count': 31,
  'row_count': 36947},
 {'file': 'detci_12_15_january_2025.xls',
  'sheet': 'nextech',
  'sheet_count': 1,
  'column_count': 31,
  'row_count': 36499},
 {'file': 'detci_16_18_january_2025.xls',
  'sheet': 'nextech',
  'sheet_count': 1,
  'column_count': 31,
  'row_count': 35982},
 {'file': 'detci_19_22_january_2025.xls',
  'sheet': 'nextech',
  'sheet_count': 1,
  'column_count': 31,
  'row_count': 39830},
 {'file': 'detci_23_25_january_2025.xls',
  'sheet': 'nextech',
  'sheet_count': 1,
  'column_count': 31,
  'row_count': 35111},
 {'file': 'detci_27_29_january_2025.xls',
  'sheet': 'nextech',
  'sheet_count': 1,
  'column_cou

In [ ]:
# TODO: make sure the number of rows read from excel matches the number of rows written to parquet and logged in the database

In [6]:
# get C:\etl\duckdb\bronze_layer.db tables
con2 = duckdb.connect(database="duckdb/bronze_layer.db")
schemas = con2.execute("SHOW TABLES").fetchall()
schemas


[('bronze_sales_cash_invoices_detailed',),
 ('bronze_sales_cash_invoices_summarized',)]

In [7]:
# check for negative values in the bronze_sales_cash_invoices_detailed
con2.close()